In [66]:
import pandas as pd
import numpy as np
import warnings
import os
pd.set_option('display.max_rows', 1250)

In [67]:
column_names_threat=['Recorded_Time','Received_Time','Serial_Number','Type','Subtype', 'Config_Version','Generated_Time',
                     'Source_IP','Destination_IP','NAT_Source_IP','NAT_Destination_IP','Rule_Name','Source_User','Destination_User',
                     'Application','Virtual_System','Source_Zone','Destination_Zone','Ingress_Interface','Egress_Interface', 
                     'Log_Forwarding_Profile','Future_Use','Session_ID','Repeat_Count','Source_Port','Destination_Port', 
                     'NAT_Source_Port','NAT_Destination_Port','Flags','Protocol','Action', 'Miscellaneous','Threat_ID', 
                     'Category', 'Severity', 'Direction','Sequence_Number', 'Action_Flags','Source_Location','Destination_Location',
                     'Drop_40','Drop_41','Drop_42','Drop_43','Drop_44','Something','Drop_46','Drop_47','Drop_48','Drop_49',
                     'Drop_50','Drop_51','Drop_52','Drop_53','Drop_54','Drop_55','Drop_56','Drop_57','Drop_58',
                     'device_name','Drop_60','Drop_61','Drop_62','Drop_63','Drop_64','Drop_65','Drop_66','Drop_67','Drop_68',
                     'Detailed_Subtype','Something_2','From_Policy']

In [68]:
def read_logs(rel_path,compression=None,extension=None,column_names=None):
    path = rel_path
    zip_files = [f for f in os.listdir(path) if f.endswith(extension)]
    paths = [path + x for x in zip_files]
    df=pd.DataFrame()
    for i in range(0,len(paths)):
        try:
            demo_df=pd.read_csv(paths[i],header=None,infer_datetime_format=True,low_memory=True,
                                compression=compression,error_bad_lines=False,warn_bad_lines=False,memory_map=True)
        except:
#             print(i,paths[i],"\n")
            continue
        if demo_df.empty!=True:
            df=df.append(demo_df)
        else:
            print("Not empty")
    df.columns=column_names
    return df

In [69]:
root_paths=['../../data/2017-09-26/','../../data/2017-09-27/','../../data/2017-09-28/','../../data/2017-09-29/',
            '../../data/2017-09-30/','../../data/2017-10-01/','../../data/2017-10-02/','../../data/2017-10-03/',
            '../../data/2017-10-04/','../../data/2017-10-05/','../../data/2017-10-06/','../../data/2017-10-07/',
            '../../data/2017-10-08/','../../data/2017-10-09/','../../data/2017-10-10/','../../data/2017-10-11/'
           ,'../../data/2017-10-12/','../../data/2017-10-13/','../../data/2017-10-14/','../../data/2017-10-15/'
           ,'../../data/2017-10-16/','../../data/2017-10-17/','../../data/2017-10-18/','../../data/2017-10-19/'
           ,'../../data/2017-10-20/','../../data/2017-10-21/','../../data/2017-10-22/','../../data/2017-10-23/'
            ,'../../data/2017-10-24/','../../data/2017-10-25/','../../data/2017-10-26/']

threat_logs=pd.DataFrame()
for i in range(0,len(root_paths)):
    df2=(read_logs(root_paths[i],None,'Wireless-PA.csv',column_names_threat))
    threat_logs=pd.concat([threat_logs,df2])
threat_logs.Received_Time=pd.to_datetime(threat_logs.Received_Time)
threat_logs.sort_values(by='Received_Time',inplace=True)
threat_logs.dropna(inplace=True,subset=['Source_User'])
threat_logs.reset_index(inplace=True,drop=True)

In [70]:
threat_logs.drop(['Drop_40','Drop_41','Drop_42','Drop_43','Drop_44','Drop_46','Drop_47','Drop_48','Drop_49',
                 'Drop_50','Drop_51','Drop_52','Drop_53','Drop_54','Drop_55','Drop_56','Drop_57','Drop_58',
                 'Drop_60','Drop_61','Drop_62','Drop_63','Drop_64','Drop_65','Drop_65','Drop_66','Drop_67',
                 'Drop_68',
                 'NAT_Source_IP','NAT_Destination_IP','Rule_Name','Destination_User','Virtual_System',
                 'Log_Forwarding_Profile','NAT_Source_Port','NAT_Destination_Port','Action_Flags','From_Policy',
                 'Flags','Serial_Number','Config_Version','Source_Zone','Destination_Zone','Ingress_Interface',
                'Egress_Interface'],
                axis=1,inplace=True)

In [71]:
d=threat_logs.groupby([threat_logs.Source_User,threat_logs.Severity]).count().unstack().fillna(0)['Subtype']
c=pd.DataFrame(d)
c.reset_index(inplace=True,drop=False)
c['Count']=c.critical+c.high+c.medium
c.sort_values(by='Count',ascending=False,inplace=True)
c.Count=c.Count.astype(int)
c.reset_index(inplace=True,drop=True)
x=c[c.Count<=1]
y=c[c.Count>1]

In [72]:
c.to_csv('Source_User_Severity_Count.csv')

In [73]:
c=c[c.Source_User!='bb55d0daf6d89297b9a3af725850bb5c387364a97983b7486270e590177a885b']
c=c[c.Source_User!='d0e4a472a56ed031ed5063b27df8d14e7c55fa68c35995364be2091f1230945a']
c=c[c.Source_User!='7c1c60bc43c7c095891ddf09dc36e613cfd05d98c7cde555617690d99888591f']
c=c[c.Source_User!='71486787ced3b5fcec52ba3d3a0a58d01f0814ca81abfa669834d7d5741b8fb1']

In [75]:
c.sort_values(by='critical',ascending=False)

Severity,Source_User,critical,high,medium,Count
27,1510909c55f8994397a5aced5728fca609aff1b6bab310...,44.0,0.0,0.0,44
66,1ed642700929c99329b1ec5e6ddaa5b55db8ec04f776a6...,17.0,0.0,0.0,17
90,cff22bd1423ddbbbf00818b3d866cb4a5f4a8603ee607a...,13.0,0.0,0.0,13
0,e0027ef4773a01094339852b6b1207f38c22adbe46879a...,2.0,42.0,1195.0,1239
12,eee73a52c55dbef6c5b4ec7ec0cd3d200a90f528b7344b...,2.0,72.0,17.0,91
879,4121c37e4f4081f7e533940466b0962159f00840b14dca...,1.0,0.0,0.0,1
758,db663ee7c30f6e366215d111f745205c73789819f57810...,1.0,0.0,0.0,1
1152,5672b291263e608577995d50cd51efd4662af2b9a3d327...,1.0,0.0,0.0,1
1141,5b2109fa7d9154dc8b15a1ecee9f520330519e9e4d8ea5...,1.0,0.0,0.0,1
814,3d59e8325f79d68d9b843556e4ce185e42b7d2da08db20...,0.0,0.0,1.0,1


In [76]:
print("Total number of users:",len(c))
print("Number of users who only created high severity logs:",len(c[(c.critical==0)  & (c.medium==0)]))
print("Number of users who only created medium severity logs:",len(c[(c.critical==0)  & (c.high==0)]))
print("Number of users who only created critical severity logs:",len(c[(c.medium==0)  & (c.high==0)]))

Total number of users: 1215
Number of users who only created high severity logs: 21
Number of users who only created medium severity logs: 1182
Number of users who only created critical severity logs: 7


In [77]:
print("Number of users who have created atleast one critical severity log:",len(c[c.critical>0]))
print("Number of users who have created atleast one high severity log:",len(c[c.high>0]))
print("Number of users who have created atleast one medium severity log:",len(c[c.medium>0]))

Number of users who have created atleast one critical severity log: 9
Number of users who have created atleast one high severity log: 26
Number of users who have created atleast one medium severity log: 1187


In [84]:
print("Number of users who created both critical and high severity logs:",len(c[(c.critical>0) & (c.high>0)]))
print("Number of users who created both critical and medium severity logs:",len(c[(c.critical>0) & (c.medium>0)]))
print("Number of users who created both high and medium severity logs:",len(c[(c.medium>0) & (c.high>0)]))
print("Number of users who created atleast one logs of each type:",len(c[(c.medium>0)&(c.high>0)&(c.critical>0)]))

Number of users who created both critical and high severity logs: 2
Number of users who created both critical and medium severity logs: 2
Number of users who created both high and medium severity logs: 5
Number of users who created atleast one logs of each type: 2


In [89]:
print("Percent of users who created only medium severity logs:",len(c[(c.critical==0)  & (c.high==0)])/len(c)*100,"%")
print("Percent of users who created only high severity logs:",len(c[(c.critical==0)  & (c.medium==0)])/len(c)*100,"%")
print("Percent of users who created only critical severity logs:",len(c[(c.high==0)  & (c.medium==0)])/len(c)*100,"%")

Percent of users who created only medium severity logs: 97.28395061728395 %
Percent of users who created only high severity logs: 1.728395061728395 %
Percent of users who created only critical severity logs: 0.5761316872427984 %
